In [6]:
### Assignment 1 in DAT510. Cryptanalysis of primitive ciphers ###
### Part 2. Simplified TripleDES

from sys import exit
class TripleSDES:

    key_length = 10
    subkey_length = 8

    P10 = (3,5,2,7,4,10,1,9,8,6)
    P8 = (6,3,7,4,8,5,10,9)
    IP = (2,6,3,1,4,8,5,7)
    Ipinverse = (4,1,3,5,7,2,8,6)
    EP = (4,1,2,3,2,3,4,1)
    S0 = (1,0,3,2,3,2,1,0,0,2,1,3,3,1,3,2)
    S1 = (0,1,2,3,2,0,1,3,3,0,1,0,2,1,0,3)
    P4 = (2,4,3,1)
    
  
    def perm(self, inputByte, permTable):
        outputByte = 0
        for index, elem in enumerate(permTable):
            if index >= elem:
                outputByte |= (inputByte & (128 >> (elem - 1))) >> (index - (elem - 1))
            else:
                outputByte |= (inputByte & (128 >> (elem - 1))) << ((elem - 1) - index)
        return outputByte
    
    def ip(self, inputByte):
        return self.perm(inputByte, self.IP)
    
    def Ipinverse_func(self, inputByte):
        return self.perm(inputByte, self.Ipinverse)
    
    def swapNibbles(self, inputByte):
        return (inputByte << 4 | inputByte >> 4) & 0xff
    
    def leftShift(keyBitList):
        shiftedKey = [None] * self.key_length
        shiftedKey[0:9] = keyBitList[1:10]
        shiftedKey[4] = keyBitList[0]
        shiftedKey[9] = keyBitList[5]
        return shiftedKey
    
    def keyGen(self, key):
        def leftShift(keyBitList):
            shiftedKey = [None] * self.key_length
            shiftedKey[0:9] = keyBitList[1:10]
            shiftedKey[4] = keyBitList[0]
            shiftedKey[9] = keyBitList[5]
            return shiftedKey
        
        # Converts input key (integer) into a list of binary digits
        keyList = [(key & 1 << i) >> i for i in reversed(range(self.key_length))]
        permKeyList = [None] * self.key_length
        for index, elem in enumerate(self.P10):
            permKeyList[index] = keyList[elem - 1]
        shiftedOnceKey = leftShift(permKeyList)
        shiftedTwiceKey = leftShift(leftShift(shiftedOnceKey))
        subKey1 = subKey2 = 0
        for index, elem in enumerate(self.P8):
            subKey1 += (128 >> index) * shiftedOnceKey[elem - 1]
            subKey2 += (128 >> index) * shiftedTwiceKey[elem - 1]
        return (subKey1, subKey2)
    
    def fk(self, subKey, inputData):
        def F(sKey, rightNibble):
            aux = sKey ^ self.perm(self.swapNibbles(rightNibble), self.EP)
            index1 = ((aux & 0x80) >> 4) + ((aux & 0x40) >> 5) + \
                    ((aux & 0x20) >> 5) + ((aux & 0x10) >> 2)
            index2 = ((aux & 0x08) >> 0) + ((aux & 0x04) >> 1) + \
                    ((aux & 0x02) >> 1) + ((aux & 0x01) << 2)
            sboxOutputs = self.swapNibbles((self.S0[index1] << 2) + self.S1[index2])
            return self.perm(sboxOutputs, self.P4)
    
        leftNibble, rightNibble = inputData & 0xf0, inputData & 0x0f
        return (leftNibble ^ F(subKey, rightNibble)) | rightNibble
    
    def encrypt(self, key, plaintext):
        data = self.fk(self.keyGen(key)[0], self.ip(plaintext))
        return self.Ipinverse_func(self.fk(self.keyGen(key)[1], self.swapNibbles(data)))
    
    def decrypt(self, key, ciphertext):
        data = self.fk(self.keyGen(key)[1], self.ip(ciphertext))
        return self.Ipinverse_func(self.fk(self.keyGen(key)[0], self.swapNibbles(data)))  
    
# Instance of Class Main 
Object = TripleSDES() 
  
choice = ''
while choice != "0":
    choice = input('Enter e/E for Encrypt, d/D for Decrypt and 0 to exit: ')
    if choice.lower()=='e':
        k1 = int(input("Input binary value of Raw Key k1: "), 2)
        k2 = int(input("Input binary value of Raw Key k2: "), 2)
        pt = int(input("Input binary value of Plaintext: "), 2)
        print("encrypted result: ", bin(Object.encrypt(k1,Object.decrypt(k2, Object.encrypt(k1,pt)))))
    elif choice.lower()=='d':
        k1 = int(input("Input binary value of Raw Key k1: "), 2)
        k2 = int(input("Input binary value of Raw Key k2: "), 2)
        ct = int(input("Input binary value of Ciphertext: "), 2)
        print("decrypted result: ", bin(Object.decrypt(k1,Object.encrypt(k2, Object.decrypt(k1,ct)))))
    elif choice != '0':
        print ('You have entered an invalid input, please try again. \n\n')
exit()

Enter e/E for Encrypt, d/D for Decrypt and 0 to exit: e
Input binary value of Raw Key k1: 1111111111
Input binary value of Raw Key k2: 1111111111
Input binary value of Plaintext: 00000000
encrypted result:  0b11101011
Enter e/E for Encrypt, d/D for Decrypt and 0 to exit: 0


SystemExit: 

D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
